In [147]:
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
# guide
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal

In [149]:
perf_variance = 5.0

def model(player_parameters, matches, outcomes=None):

    skill_mu = player_parameters[0]
    skill_sigma = player_parameters[1]

    num_players = len(skill_mu)

    with pyro.plate("players", num_players):
        skill = pyro.sample("skill", dist.Normal(skill_mu, skill_sigma))

    with pyro.plate("games", num_games):

        # Get player skills
        player1_skill = skill[matches[:, 0]]
        player2_skill = skill[matches[:, 1]]

        # Sample performances
        player1_performance = pyro.sample("perf1", dist.Normal(player1_skill, perf_variance))
        player2_performance = pyro.sample("perf2", dist.Normal(player2_skill, perf_variance))

        # Sample outcomes
        result = pyro.sample("obs", dist.Bernoulli(logits=player1_performance - player2_performance), obs=outcomes)

    return result


# Guide
def guide(parameters, players, outcomes=None):

    num_players = len(parameters[0])

    skill_loc = pyro.param("skill_loc", torch.zeros(num_players))
    skill_scale = pyro.param("skill_scale", torch.ones(num_players), constraint=dist.constraints.positive)

    with pyro.plate("players", num_players):
        skill = pyro.sample("skill", dist.Normal(skill_loc, skill_scale))

    with pyro.plate("games", num_games):

        # Get player skills
        player1_skill = skill[matches[:, 0]]
        player2_skill = skill[matches[:, 1]]

        # Sample performances
        player1_performance = pyro.sample("perf1", dist.Normal(player1_skill, perf_variance))
        player2_performance = pyro.sample("perf2", dist.Normal(player2_skill, perf_variance))


num_games = 10
num_players = 5

players = torch.randint(0, num_players, (num_games, 2))
outcomes = torch.ones(num_games)

parameters = [torch.ones(num_players) * 100., torch.ones(num_players) * 25.]

model(parameters, players)


# Fit the model
optim = Adam({"lr": 0.05})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

for step in range(1000):
    loss = svi.step(parameters, players, outcomes)
    if step % 100 == 0:
        print("Step {}: Loss = {:.2f}".format(step, loss))

print(pyro.get_param_store().get_state()['params'])


Step 0: Loss = 51.66
Step 100: Loss = 10.55
Step 200: Loss = 25.12
Step 300: Loss = 51.73
Step 400: Loss = 38.07
Step 500: Loss = 44.25
Step 600: Loss = 50.26
Step 700: Loss = 44.85
Step 800: Loss = 35.38
Step 900: Loss = 48.13
{'skill_loc': tensor([89.0027, 85.2201, 88.3698, 84.0022, 80.9570], requires_grad=True), 'skill_scale': tensor([0.3517, 0.1540, 0.5880, 0.2010, 1.0358], requires_grad=True)}
